In [1]:
import math
import random
import numpy as np
from typing import Union

In [2]:
class tenpy:

    def __init__ (self,data,prev = (),op = '',label = ''):
        self.data = data
        self.type = type(data)
        self.prev = set(prev)
        self._backword = lambda: None
        self.op = op
        self.label = label
        self.grad = 0


    def __repr__(self):

        return f"tenpy(data  = {self.data}   {self.type})"
    
    def __add__(self,other):

        other = other if isinstance(other,tenpy) else tenpy(other)

        new =  tenpy( self.data + other.data , (self,other),'+')

        def _backword():

            self.grad += 1.0 *new.grad
            other.grad += 1.0 *new.grad

        new._backword = _backword
        return new
        



    def __radd__(self,other):
         
         return self+other




    def __mul__(self,other):

        other = other if isinstance(other,tenpy) else tenpy(other)

        
        new = tenpy( self.data * other.data , (self,other),'*') 
        def _backword():
            self.grad +=  other.data *new.grad
            other.grad += self.data *new.grad
        
        new._backword = _backword

 
        return new
    
    def __rmul__(self,other):
        return self * other
    


    def __truediv__(self,other):
        return self*(other**-1)
    


    def __pow__(self,other):

        assert isinstance(other,(float,int))

        new = tenpy(self.data**other,(self,),f'**{other}')

        def _backword():
            self.grad +=  other *(self.data**(other -1)) * new.grad    
        
        new._backword = _backword
        

        
        return new


    def __rtruediv__(self, other):
        return (self**-1)*other

    def __sub__(self,other):
        return self +(-other)
    
    def __neg__(self):
        return self*-1
    
    def exp(self):

        new = tenpy(math.exp(self.data),(self,),"exp")

        def _backword():

            self.grad += new.data * new.grad

        new._backword = _backword

        return new
    
    def tanh(self):

        x2 =  2*self.data
        ex2 =  math.exp(x2)
        sinh = ex2 - 1
        cosh = ex2 + 1
        new = tenpy(sinh/cosh , (self,),"tanh")

        def _backword():
           
            self.grad +=  (1 -new.data**2)*new.grad

        new._backword = _backword
        return new  
    
    def backword(self):

        topo = []
        visited = set()
        def build_topo(val):
            if val not in visited:
                visited.add(val)
                for  child in val.prev:
                    
                    build_topo(child)
                topo.append(val)
        
        build_topo(self)
        self.grad = 1
        for child in reversed(topo):
            child._backword()
    

In [3]:
from graphviz import Digraph

def trace(val):
    nodes = set()
    edges = set()
    def get(v):
        if v not in nodes:
            nodes.add(v)
            for i in v.prev:
                edges.add((i,v))
                get(i)
    get(val)
    return nodes,edges

def visuvalize(val):

    plot = Digraph(format='svg',graph_attr= {'rankdir':'LR'})

    nodes , edges = trace(val)

    for node in nodes:

        uid = str(id(node))
        plot.node( name = uid, label= f"{node.label}|data = {node.data}|grad = {node.grad}", shape = 'record')

        if node.op:
            plot.node(name = uid+ node.op , label= node.op)
            plot.edge(uid+node.op , uid)

    for n1,n2 in edges:
        plot.edge(str(id(n1)),str(id(n2))+n2.op)

    return plot


    

In [4]:
class Neuron:
    def __init__(self,inp):
        self.W = [tenpy(random.uniform(-1,1)) for i in range(inp)]
        self.b = tenpy(random.uniform(-1,1))
        self.inp = inp
    def __call__(self,x):
        
        act =sum((wi*xi for wi, xi in zip(self.W, x)), self.b)
        
        return(act.tanh())
    
    def parameter(self):

        return  self.W + [self.b]
    
class Layer:

    def __init__(self,inp,op):

        self.neurons = [Neuron(inp) for i in range(op)]
    
    def __call__(self,x):

        outs = [neuron(x) for neuron in self.neurons]

        return outs[0] if len(outs)==1 else outs
    
    def parameter(self):

        return [p for neuron in self.neurons for p in neuron.parameter()]
    
class MLP:

    def __init__(self,inp,ops):

        N = [inp] + list(ops)

        self.layers = [Layer(N[i],N[i+1]) for i in range(len(N) - 1)]

    def __call__(self,x):

        for layer in self.layers:

            x = layer(x)

        return x
    
    def parameter(self):

        return [p for layer in self.layers for p in layer.parameter()]
    
    def reset_grade(self):

        for p in self.parameter():

            p.grad = 0

    def update_weights(self):

        for p in self.parameter():

            p.data += -0.1*p.grad

    def fit(self,xs,ys,epochs):

        for i in range(epochs):

            ypred = [self(x) for x in xs]
            loss = sum((yp - y)**2 for yp,y in zip(ypred,ys))
            self.reset_grade()
            loss.backword()
            self.update_weights()
            print(loss)

    def predict(self,xs: list[list[(int,float)]]):

        mypred  = [self(x) for x in xs]
        
        return mypred










In [5]:
xs = [
  [2.0, 3.0, -1.0],
  [3.0, -1.0, 0.5],
  [0.5, 1.0, 1.0],
  [1.0, -1.0, -1.0],
 [1.0, 1.0, -1.0] ,
]
ys = [1.0, -1.0, -1.0, 1.0,-1] # desired targets

len1 = len(ys)

In [6]:
n = MLP(3,[4,4,5,1])

In [7]:
n.fit(xs,ys,100)

tenpy(data  = 7.334242603261702   <class 'float'>)
tenpy(data  = 4.761633944073488   <class 'float'>)
tenpy(data  = 9.274724053843192   <class 'float'>)
tenpy(data  = 3.6160108650057694   <class 'float'>)
tenpy(data  = 4.2320353771965316   <class 'float'>)
tenpy(data  = 5.948517981181035   <class 'float'>)
tenpy(data  = 3.0143589487500977   <class 'float'>)
tenpy(data  = 2.837997355771375   <class 'float'>)
tenpy(data  = 2.95584353133582   <class 'float'>)
tenpy(data  = 2.733836417483152   <class 'float'>)
tenpy(data  = 2.818911348660561   <class 'float'>)
tenpy(data  = 2.6811204236020965   <class 'float'>)
tenpy(data  = 3.2530369378142434   <class 'float'>)
tenpy(data  = 2.2310473435205234   <class 'float'>)
tenpy(data  = 3.011408939002864   <class 'float'>)
tenpy(data  = 3.544471554006099   <class 'float'>)
tenpy(data  = 5.12294450814724   <class 'float'>)
tenpy(data  = 3.575299293975994   <class 'float'>)
tenpy(data  = 3.3036331166153805   <class 'float'>)
tenpy(data  = 2.9512613640

In [8]:
n.predict(xs)

[tenpy(data  = 0.2820951378817281   <class 'float'>),
 tenpy(data  = -0.9621524470623408   <class 'float'>),
 tenpy(data  = -0.9709960711369958   <class 'float'>),
 tenpy(data  = 0.2933238091059199   <class 'float'>),
 tenpy(data  = 0.27701423749114307   <class 'float'>)]

In [9]:
n.parameter()[0].grad

0.056168177404032196

In [10]:
n.reset_grade()
n.parameter()[0].grad

0